# VacationPy
----
    
Using the weather dataset of the cities, we see the the heatmap of humidity of the cities. We select a few cities of which weather satisfy the vacation weather criteria, and then mark the nearby hotels on the heatmap.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import json
import gmaps

# Import API key
from api_keys import g_key

### Load Data 
* Load the csv exported in Part I (the weather information of the cities) to a DataFrame

In [2]:
weather_df = pd.read_csv("output_data/cities_weathers.csv")
weather_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed
0,5106841,Albany,US,1594699812,42.60,-73.97,73.004,71,32,3.34
1,4031574,Provideniya,RU,1594700035,64.38,-173.30,50.000,81,90,6.00
2,3831208,Qaanaaq,GL,1594699991,77.48,-69.36,41.522,72,1,1.24
3,5122534,Jamestown,US,1594699977,42.10,-79.24,62.600,82,1,2.10
4,686090,Asău,RO,1594700083,46.43,26.40,57.200,71,19,1.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Set the locations with Lattitude and Longitude
locations = weather_df[['Lat','Lng']]

# Set the weights as the humidity
humidities = weather_df['Humidity']

# Creat the base map centered with the Raleigh coordinate
Raleigh_coord = (35.7796, -78.6382)
wfig = gmaps.figure(center=Raleigh_coord, zoom_level=2)

# Set the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidities,
                                dissipating=False, max_intensity=10,
                                point_radius=1)

# Add the layer
wfig.add_layer(heat_layer)

# Show the gmap
wfig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.

In [59]:
# Weather criteria
vac_weather_df = weather_df.loc[lambda x: \
                             (x['Max Temp (F)'] < 78) & \
                             (x['Humidity'] < 35)     & \
                             (x['Cloudiness'] < 30)   & \
                             (x['Wind Speed'] < 10)]
vac_weather_df

,City ID,City,Country,Date,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Wind Speed
102,5773664,Davis,US,1594700089,41.00,-112.00,77.0,31,0,5.11
209,347634,Ţahţā,EG,1594700096,26.77,31.50,77.0,31,0,3.60
214,5710360,Winnemucca,US,1594700009,40.97,-117.74,73.4,15,1,1.50
379,347796,Sohag,EG,1594700084,26.56,31.69,77.0,31,0,3.60
380,114049,Sonqor,IR,1594700113,34.78,47.60,73.4,27,7,0.23
406,1526041,Atasū,KZ,1594700115,48.68,71.64,75.2,34,0,7.04
435,5493811,Taos,US,1594700118,36.41,-105.57,75.2,33,1,5.70
445,5783695,Tooele,US,1594699845,40.53,-112.30,77.0,33,1,3.54
463,2077895,Alice Springs,AU,1594699913,-23.70,133.88,60.8,30,0,6.70
492,2068110,Kununurra,AU,1594700122,-15.77,128.73,75.2,16,0,5.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
# Store variable in hotel_df
hotel_df = vac_weather_df[['City','Country','Lat','Lng']].reset_index(drop=True)

In [65]:
# Set the base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set up a parameters dict
params = {
    "radius" : 5000,
    "type"   : "lodging",
    "key"    : g_key
}

# Initialize the lists to hold the hotel informations
hotel_names = []
hotel_lats = []
hotel_lngs = []

# Run requests with the selected cities locations
for i in hotel_df.index:
    # get a city location
    lat = hotel_df.at[i, 'Lat']
    lng = hotel_df.at[i, 'Lng']
    
    # add the location field to the parameter
    params["location"] = f"{lat}, {lng}"
    
    # request hotel data
    response = requests.get(base_url, params=params)
    hotel_data = response.json()

    # update the hotel info lists
    try:
        hotel_names.append(hotel_data["results"][0]["name"])
        hotel_lats.append(hotel_data["results"][0]["geometry"]["location"]["lat"])
        hotel_lngs.append(hotel_data["results"][0]["geometry"]["location"]["lng"])
    except:
        hotel_names.append(None)
        hotel_lats.append(None)
        hotel_lngs.append(None)
        print(f"Hotels Not found in {hotel_df.at[i, 'City']}")
        pass

Hotels Not found in Davis


In [67]:
# Add the columns for hotel information
hotel_df['Hotel Name'] = hotel_names
hotel_df['Hotel Lat'] = hotel_lats
hotel_df['Hotel Lng'] = hotel_lngs

# Remove the cities where the hotels are not found
hotel_df = hotel_df.dropna()

hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Lat,Hotel Lng
1,Ţahţā,EG,26.77,31.50,Møhãmęd Tãręk 🏠,26.772038,31.496757
2,Winnemucca,US,40.97,-117.74,Winners Inn Casino,40.972023,-117.735982
3,Sohag,EG,26.56,31.69,Civil Registry - Sohag 1,26.546997,31.701741
4,Sonqor,IR,34.78,47.60,روستای خسروآباد امجدی,34.777894,47.596328
5,Atasū,KZ,48.68,71.64,Dostyk,48.688509,71.687949
6,Taos,US,36.41,-105.57,Inn on La Loma Plaza,36.405804,-105.580975
7,Tooele,US,40.53,-112.30,Holiday Inn Express & Suites Tooele,40.559119,-112.297444
8,Alice Springs,AU,-23.70,133.88,Desert Palms Alice Springs,-23.713395,133.879820
9,Kununurra,AU,-15.77,128.73,Hotel Kununurra,-15.776260,128.738450


In [68]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[['Lat', 'Lng']]

In [69]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
wfig.add_layer(markers)

# Display figure
wfig

Figure(layout=FigureLayout(height='420px'))